In [254]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
       print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session




/kaggle/input/training-data/training_data.pickle
/kaggle/input/painter-by-numbers/all_data_info.csv
/kaggle/input/painter-by-numbers/submission_info.csv
/kaggle/input/painter-by-numbers/train.zip
/kaggle/input/painter-by-numbers/train_info.csv
/kaggle/input/painter-by-numbers/train_3.zip
/kaggle/input/painter-by-numbers/train_2.zip
/kaggle/input/painter-by-numbers/sampleSubmission.csv
/kaggle/input/painter-by-numbers/train_6.zip
/kaggle/input/painter-by-numbers/train_5.zip
/kaggle/input/painter-by-numbers/train_8.zip
/kaggle/input/painter-by-numbers/solution_painter.csv
/kaggle/input/painter-by-numbers/train_7.zip
/kaggle/input/painter-by-numbers/test.zip
/kaggle/input/painter-by-numbers/train_4.zip
/kaggle/input/painter-by-numbers/replacements_for_corrupted_files.zip
/kaggle/input/painter-by-numbers/train_1.zip
/kaggle/input/painter-by-numbers/train_9.zip
/kaggle/input/notebook333c9b5d70/training_data_out
/kaggle/input/notebook333c9b5d70/__results__.html
/kaggle/input/notebook333c9b5d

In [257]:
#Creating the training data 
import zipfile
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np 

data = pd.read_csv('/kaggle/input/painter-by-numbers/train_info.csv')
relevant_style = 'Cubism'
relevant_data = data[data['style'] == relevant_style]

Dataset = 'painter-by-numbers/train'

zip_path = '../input/painter-by-numbers/train.zip'
relevant_files = [str("train/" + i) for i in relevant_data['filename']]

print(len(relevant_data))



1316


In [258]:
Image.MAX_IMAGE_PIXELS = None
training_data = None
count = 0
with zipfile.ZipFile(zip_path) as myzip:
    fileInitial = myzip.open(relevant_files[0])
    img = Image.open(fileInitial)
    img = img.resize((64, 64))
    training_data = np.array([np.asarray(img)])
    for file in relevant_files[1:]:
        
        with myzip.open(file) as myfile:
            print(count)
            count += 1

            img = Image.open(myfile)
            img = img.resize((64, 64))
            if (np.asarray(img).shape == (64, 64)):
                continue
            elif (np.asarray(img).shape == (64, 64, 4)):
                continue
            
            
            training_data = np.append(training_data, np.array([np.asarray(img)]), axis=0)
training_data.shape
        

0


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 200 and the array at index 1 has size 64

In [ ]:
training_data.shape
with zipfile.ZipFile(zip_path) as myzip:
    fileInitial = myzip.open(relevant_files[5274])
    img = Image.open(fileInitial)
    img = img.resize((200, 200))
img

In [ ]:
len(relevant_files)

In [ ]:
import pickle
filename = '../input/training-data/training_data.pickle'

    
training_data = None
with open(filename, 'rb') as infile:
    training_data = pickle.load(infile)





IMAGE_SIZE = 64

def normalize(img):
  
  img = tf.cast(img, tf.float32)
  img = (img / 127.5) - 1
  return tf.image.resize(img, (IMAGE_SIZE, IMAGE_SIZE))

def processfile(image):

    image = normalize(image)
    
    return tf.expand_dims(image[:, :, 0], axis=2)



In [ ]:


normalized_data = np.array([processfile(i) for i in training_data])
print(normalized_data.shape)
print(normalized_data)


In [ ]:
import tensorflow as tf



import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time

from IPython import display


normalized_data = normalized_data[7500:]


BUFFER_SIZE = normalized_data.shape[0]


BATCH_SIZE = 75
train_dataset = tf.data.Dataset.from_tensor_slices(normalized_data).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
BUFFER_SIZE

plt.imshow(normalized_data[0], cmap='gray')
plt.colorbar()

print(normalized_data[0])

In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(8*8*256, use_bias=False, input_shape=(150,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((8,8, 256)))
    assert model.output_shape == (None, 8, 8, 256)  # Note: None is the batch size
    
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 16, 16, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 32, 32, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    
    assert model.output_shape == (None, 64, 64, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(1, 1), padding='same', use_bias=False, activation='tanh'))
    
    assert model.output_shape == (None, 64, 64, 1)

    return model
generator = make_generator_model()
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[64, 64, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model
discriminator = make_discriminator_model()


In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

EPOCHS = 200
noise_dim = 150
num_examples_to_generate = 16

# You will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images, print_error=False):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)
      if print_error == True:
            
          tf.print("Generator loss was ",gen_loss)
          tf.print("Discriminator loss was ", disc_loss)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()
    counter = 0

    for image_batch in dataset:
      counter += 1
      train_step(image_batch, print_error = (counter % 10 == 0) )

    # Produce images for the GIF as you go
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           seed)
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(4, 4))
  #predictions = predictions*127.5 + 127.5

  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0], cmap='gray')
      plt.axis('off')

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.colorbar()
  plt.show()


In [ ]:
image_batch = next(iter(train_dataset))
image_batch.shape

In [ ]:
import matplotlib.pyplot as plt
train(train_dataset, EPOCHS)

In [ ]:
test = tf.random.normal([num_examples_to_generate, noise_dim])
generating = generator(test, training=False)
fake_output = discriminator(generating, training=True)


print(generating.shape)
print(fake_output)
firstimage = generating[2]
print(firstimage.shape)


plt.imshow(firstimage, cmap='gray')
plt.colorbar()

In [ ]:
plt.imshow(generating[0], cmap='gray')

In [ ]:
def display_image(epoch_no):
  return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))
display_image(EPOCHS)

anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)
import tensorflow_docs.vis.embed as embed
embed.embed_file(anim_file)

In [ ]:
generator.save('/generator')
discriminator.save('/discriminator')

In [ ]:
tf.executing_eagerly()